<a href="https://colab.research.google.com/github/ChelseaOwiti/Colposcopy-Scan/blob/main/Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#coding resnet architecture


In [2]:
import torch
import torch.nn as nn

In [3]:
#creating blocks
class block(nn.Module):
  def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
    super(block, self).__init__()
    self.expansion = 4
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 1, stride=1, padding=0)
    #Batch Norm 
    self.bn1 = nn.BatchNorm2d(out_channels)
    self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride = stride, padding = 1)
    self.bn2 = nn.BatchNorm2d(out_channels)
    #Another conv
    self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size= 1 , stride = 1, padding=0)
    self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)

    #Relu Layer
    self.relu =  nn.ReLU()
    self.identity_downsample = identity_downsample


  def forward(self, x):
    identity = x

    x = self.conv1(x)
    x =  self.bn1(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)
    x = self.conv3(x)
    x = self.bn3(x)

# incase we need to change shape
    if self.identity_downsample is not None:
      identity = self.identity_downsample(identity)

      x += identity
      x = self.relu(x)
      return x





In [5]:
class ResNet(nn.Module):  #layers= how many times we will reuse the blocks
  def __init__(self, block, layers, image_channels, num_classes): #image_channels = no. of channels of input
    super(ResNet, self).__init__()
    self.in_channels = 64
    self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride =2, padding=3)
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU()
    self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding =1)

    #ResNet Layers out channels
    self.layer1 = self._make_layer(block, layers[0], out_channels=64, stride =1)
    self.layer2 = self._make_layer(block, layers[1], out_channels=128, stride =2)
    self.layer3 = self._make_layer(block, layers[2], out_channels=256, stride =2)
    self.layer4 = self._make_layer(block, layers[3], out_channels=512, stride =2)

    self.avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(512*4, num_classes)



  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x= self.layer1(x)
    x= self.layer2(x)
    x= self.layer3(x)
    x= self.layer4(x)

    x =self.avgpool(x)
    x = x.reshape(s.shape[0], -1)
    x = self.fc(x)

    return x


  def _make_layer(self, block, num_residual_blocks, out_channels, stride):
    identity_downsample = None
    layers = []

    if stride != 1 or self.in_channels * 4:
      identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels, out_channels*4, kernel_size=1,
                                                    stride = stride),
                                          nn.BatchNorm2d(out_channels*4))
      #this layer changes the no. of channels
    layers.append(block(self.in_channels, out_channels, identity_downsample, stride))
    self.in_channels = out_channels*4 #256

    for i in range(num_residual_blocks -1):
      layers.append(block(self.in_channels, out_channels))

    return nn.Sequential(*layers)

  def ResNet50(img_channels=3, num_classes=1000):
    return ResNet(block, [3,4,6,3], img_channels, num_classes)


  def test():
    #debug here 
    net = ResNet50()
    x = torch.randn(2, 3, 224, 224)
    y = net(x).to('Cuda')
    print(y.shape)

  test()







NameError: ignored